In [1]:
import requests, bs4, selenium, io, time
from PyPDF2 import PdfFileReader
from selenium import webdriver
#link = ["https://ocw.mit.edu/courses/11-s940-development-planning-and-implementation-the-dialectic-of-theory-and-practice-fall-2015/", "https://ocw.mit.edu/courses/11-s955-the-sustainability-response-to-covid-19-january-iap-2021/", "https://ocw.mit.edu/courses/12-007-geobiology-spring-2013/", "https://ocw.mit.edu/courses/16-810-engineering-design-and-rapid-prototyping-january-iap-2007/"]

In [2]:
PATH = './driver/chromedriver.exe'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36'
}
pref = 'https://ocw.mit.edu'

In [3]:
class CourseRes:
    formato=""
    typeR=""
    nome=""
    link=""
    durata=""

    def __init__(self, nome, link, durata, course):
        self.nome=nome
        self.formato="pdf"
        self.typeR="presentazione"
        self.link=link
        self.durata=durata
        self.course=course
    
    
    def printInCSV(self, file):
        for attr in self.__dict__:
            item = self.__dict__[attr]
            if not isinstance(item, CourseInfo):
                file.write("{};".format(item))
            else:
                #if isinstance(item[0], CourseInfo):
                #    file.write("{};".format([r.toString() for r in item]))
                file.write(item.toString())
        file.write("\n")
    '''        
    def toString(self):
        strin=""
        for attr in self.__dict__:
            item=self.__dict__[attr]
            strin= strin+"{};".format(item)
        return strin[:-1]
    '''

class CourseInfo:
    idN=""
    link=""
    discip=""
    kywTit="keyword"
    difficulty=""
    min_age=0
    max_age=0
    desc=""
    pubblico=[]
    #res=[]

    def __init__(self, link, name, idl, difficulty, topic, desc):
        self.discip=topic
        self.link=link
        self.idN=idl
        self.difficulty=difficulty
        self.desc=desc
        if "undergraduate" in difficulty.lower():
            self.pubblico = ['Middle School', 'High School']
            self.min_age=11
            self.max_age=18
        else:
            self.pubblico = ['College General Ed', 'Graduate School', 'Professional']
            self.min_age=19
            self.max_age=100
    
    def toString(self):
        strin=""
        for attr in self.__dict__:
            item=self.__dict__[attr]
            strin= strin+"{};".format(item)
        return strin

    #difficulty: graduate=medio alta, undergraduate=media
    #formato: lecture notes=pdf, assignments=pdf, 
    #durata: slidex2 = minuti {[0-30]...[120+]}, 
    #pubblico: undergrad=['Middle School', 'High School'], graduate=['College General Ed', 'Graduate School', 'Professional]
    #type: lecture notes=presentation, assignments=Quiz/Test, 
    #min_age: undergrad=11, grad=19, max_age: undergrad=18, grad=100
    '''
    def printInCSV(self, file):
        for attr in self.__dict__:
            item = self.__dict__[attr]
            if not type(item) is list:
                file.write("{};".format(item))
            else:
                if isinstance(item[0], CourseRes):
                    file.write("{};".format([r.toString() for r in item]))
                else:
                    file.write("{};".format(item))
        file.write("\n")
    '''

In [4]:
def datacollector (link, f):
    driver = webdriver.Chrome(PATH)
    driver.get(link)
    html = driver.page_source
    d = bs4.BeautifulSoup(html, "html.parser").find_all("div", {"id" : "course-description"})[0].text.replace("\n", " ").replace(";", ",")
    lvl = bs4.BeautifulSoup(html, "html.parser").find_all("a", {"class" : "course-info-level"})[0].text.replace(" ", "").replace("\n", "")
    tit = bs4.BeautifulSoup(html, "html.parser").find_all("a", {"class" : "text-uppercase display-4 font-weight-bold m-0 text-white"})[0].text.replace("\n", "")
    idl = bs4.BeautifulSoup(html, "html.parser").find_all("ul", {"class" : "list-unstyled m-0"})[1].text.replace(" ", "").replace("\n", "")
    t = bs4.BeautifulSoup(html, "html.parser").find_all("table", {"id" : "course-info-table"})
    topic = ""
    for c in t:
        for a in c.find_all("ul", {"class" : "list-unstyled pb-2 m-0"}):
            for g in a.find_all("li"):
                topic += g.text.replace("\n", "").replace ("      >          ", "/")
    course = CourseInfo(link, tit, idl, lvl, topic, d)
    patternlink= link[len(pref):]+"pages/lecture-notes"
    if bs4.BeautifulSoup(html, "html.parser").find_all("a", {"href" : patternlink}):
        nlink = pref + patternlink
        driver.quit()
        getCoursePDF(nlink, f, course) 
        del course
        print(nlink)
    

In [5]:
def getCoursePDF(link, f , course):
    coursesRes=[]
    driver = webdriver.Chrome(PATH)
    driver.get(link)
    html = driver.page_source
    soup = bs4.BeautifulSoup(html, "html.parser").find_all("div", {"id" : "main-content"})
    for s1 in soup:
        for s in s1.find_all("td"):
            for e in s.find_all("a"):
                nlink=pref+e.attrs["href"]
            
                response = requests.get(nlink,{"headers" : headers})
                href = bs4.BeautifulSoup(response.text, "html.parser").find("a", {"class":"download-file"})
                if href["href"][-3:]=="pdf":
                    pdflink = pref+href["href"]
                    response = requests.get(pdflink, stream=True)
                    my_raw_data = response.content
                
                    with open("my_pdf.pdf", 'wb') as my_data:
                        my_data.write(my_raw_data) 
                    
                        open_pdf_file = open("my_pdf.pdf", 'rb')
                        read_pdf = PdfFileReader(open_pdf_file)
                        obj=CourseRes(read_pdf.getDocumentInfo().title, pdflink, "circa {} minuti".format(read_pdf.getNumPages()), course)
                        open_pdf_file.close()
                        obj.printInCSV(f)
                        del obj
    driver.quit()

In [6]:
f = open("res_dataset.csv", "a")
links = [
    "https://ocw.mit.edu/courses/1-00-introduction-to-computers-and-engineering-problem-solving-spring-2012/",
    #"https://ocw.mit.edu/courses/1-010-uncertainty-in-engineering-fall-2008/"
    #"https://ocw.mit.edu/courses/1-011-project-evaluation-spring-2011/"
    #"https://ocw.mit.edu/courses/1-012-introduction-to-civil-engineering-design-spring-2002/"
]

for i in links:
    datacollector(i, f)
f.close()

C:\Users\Lenovo\AppData\Local\Temp/ipykernel_18252/2948007439.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
C:\Users\Lenovo\AppData\Local\Temp/ipykernel_18252/1432856309.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


PermissionError: [Errno 13] Permission denied

In [ ]:
#with io.BytesIO(my_raw_data) as open_pdf_file:
#    read_pdf = PdfFileReader(open_pdf_file)
#    num_pages = read_pdf.getNumPages()